### Universal dependencies with __[Stanza](https://stanfordnlp.github.io/stanza/#getting-started)__ and __[spacy_conll](https://spacy.io/universe/project/spacy-conll)__

In [2]:
import pandas as pd
import os
import re
import razdel
import stanza
import six
from spacy_conll import init_parser

In [4]:
news_filepath = '/Users/oksana/Dev/data/october_final.csv'
news = pd.read_csv(news_filepath, dtype={'hash':'uint64'} )
# news['hash'] = pd.util.hash_pandas_object(news.link)

news['title'] = news.title.str.replace(r'\n', ' ')
news['all_text'] = news.title.str.cat(news.text, sep='\n', na_rep = '')
news['all_text'] = news.all_text.str.strip()

news['all_text'] = news['all_text'].str.replace('не очень', 'неочень')
news['all_text'] = news['all_text'].str.replace('не дуже', 'недуже')

news.all_text = news.all_text.str.replace(r'^Редактор Цензор\.НЕТ\n', '', flags=re.M)
news.all_text = news.all_text.str.replace(r'Цензор\.НЕТ', 'Цензор')

In [3]:
stanza.download('uk')
stanza.download('ru', package='gsd', processors='tokenize,pos,lemma,depparse')

nlp = stanza.Pipeline('uk', processors='tokenize,lemma')

nlp_uk = init_parser(
        "stanza",
        "uk", 
        is_tokenized = True,
        include_headers=False,
        parser_opts = {'processors': 'tokenize,pos,lemma,depparse'}
    )

nlp_ru = init_parser(
        "stanza",
        "ru", 
        is_tokenized = True,
        include_headers=False,
        parser_opts = {'package': 'gsd', 'processors': 'tokenize,pos,lemma,depparse'}
    )

2021-01-15 13:35:57 INFO: Downloading default packages for language: uk (Ukrainian)...
2021-01-15 13:35:58 INFO: File exists: /Users/oksana/stanza_resources/uk/default.zip.
2021-01-15 13:36:03 INFO: Finished downloading models and saved to /Users/oksana/stanza_resources.
2021-01-15 13:36:03 INFO: Downloading these customized packages for language: ru (Russian)...
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| pretrain  | gsd     |

2021-01-15 13:36:03 INFO: File exists: /Users/oksana/stanza_resources/ru/tokenize/gsd.pt.
2021-01-15 13:36:03 INFO: File exists: /Users/oksana/stanza_resources/ru/pos/gsd.pt.
2021-01-15 13:36:03 INFO: File exists: /Users/oksana/stanza_resources/ru/lemma/gsd.pt.
2021-01-15 13:36:03 INFO: File exists: /Users/oksana/stanza_resources/ru/depparse/gsd.pt.
2021-01-15 13:36:04 INFO: File exists: /Users/oksana/stanza_resources/ru/pretrain/gsd.pt.
2021-01-15 13:36:04 INF

In [18]:
print(news.title.iloc[10], '\n')
doc = nlp_uk(news.title.iloc[10])
conll = doc._.conll_str
print(conll)

МОЗ оновило коронавірусні списки країн: Сім держав перейшли в зелену зону 

1	МОЗ	МОЗ	PROPN	Y	_	2	nsubj	_	_
2	оновило	оновити	VERB	Vmeis-sn	_	0	root	_	_
3	коронавірусні	коронавірусний	ADJ	Ao--pasn	_	4	amod	_	_
4	списки	список	NOUN	Ncmpan	_	2	obj	_	_
5	країн:	країна	NOUN	Ncfsgn	_	4	nmod	_	_
6	Сім	сім	NOUN	Ncfpgn	_	5	nmod	_	_
7	держав	держава	NOUN	Ncfpgn	_	6	nmod	_	_
8	перейшли	перейти	VERB	Vmeis-p	_	2	conj	_	_
9	в	в	ADP	Spsa	_	11	case	_	_
10	зелену	зелений	ADJ	Afpfsas	_	11	amod	_	_
11	зону	зона	NOUN	Ncfsan	_	8	obl	_	SpaceAfter=No



In [20]:
ACCENT = six.unichr(769)
WORD_TOKENIZATION_RULES = re.compile(r"""
[\w""" + ACCENT + """]+://(?:[a-zA-Z]|[0-9]|[$-_@.&+])+
|[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+
|[0-9]+-[а-яА-ЯіїєґІЇҐЄёЁ'’`""" + ACCENT + """]+
|[+-]?[0-9](?:[0-9,.-]*[0-9])?
|[\w""" + ACCENT + """](?:[\w'’`-""" + ACCENT + """]?[\w""" + ACCENT + """]+)*
|[\w""" + ACCENT + """].(?:\[\w""" + ACCENT + """].)+[\w""" + ACCENT + """]?
|["#$%&*+,/:;<=>@^`~…\\(\\)⟨⟩{}\[\|\]‒–—―«»“”‘’'№]
|[.!?]+
|-+
""", re.X | re.U)

ABBRS = """
ім.
в.
о.
т.
п.
д.
под.
ін.
вул.
просп.
бул.
пров.
пл.
г.
р.
див.
п.
с.
м.
""".strip().split()


def tokenize_sents(string):
    string = six.text_type(string)
    spans = []
    for match in re.finditer('[^\s]+', string):
        spans.append(match)
    spans_count = len(spans)

    rez = []
    off = 0

    for i in range(spans_count):
        tok = string[spans[i].start():spans[i].end()]
        if i == spans_count - 1:
            rez.append(string[off:spans[i].end()])
        elif tok[-1] in ['.', '!', '?', '…', '»', "'", "\""]:
            # tok1 = tok[re.search('[.!?…»]', tok).start() - 1]
            next_tok = string[spans[i + 1].start():spans[i + 1].end()]
            if (next_tok[0].isupper() or next_tok[0] in ["'", "\"", "«"]) \
                    and not ((len(tok) == 2 and tok[0].isupper()) \
                             or tok[0] == '('
                             or tok in ABBRS):
                rez.append(string[off:spans[i].end()])
                off = spans[i + 1].start()

    return rez


def text_to_sent(text, lang):
    rez = []
    if lang == 'uk':
        for part in text.split('\n'):
            rez += tokenize_sents(part)
    elif lang=='ru':
        for part in text.split('\n'):
            rez += [s.text for s in razdel.sentenize(part)]
    return rez


def sent_to_words(text, lang):
    if lang == 'uk':
        return re.findall(WORD_TOKENIZATION_RULES, text)
    elif lang == 'ru':
        return [tkn.text for tkn in razdel.tokenize(text)]
    return None


def tokenize(text, lang):
    res = []
    for sent in text_to_sent(text, lang):
        tokens = []
        for word in sent_to_words(sent, lang):
            tokens.append(word)
        res.append(' '.join(tokens))
    return '\n'.join(res)

In [22]:
print(tokenize(news.all_text.iloc[10], news.language.iloc[10]))

МОЗ оновило коронавірусні списки країн : Сім держав перейшли в зелену зону
Ілюстроване
МОЗ відносить до червоного списку ті країни , де захворюваність на 100 тис . населення вище , ніж в Україні
Міністерство охорони здоров'я України оновило червоний коронавірусний список країн .
Зокрема , в зелену зону перейшли Угорщина , Чилі , ОАЕ , Катар , Боснія і Герцеговина , Монако , Тринідад і Тобаго .
Про це свідчать дані МОЗ .
Як повідомляється , МОЗ при перегляді списків відносить до " червоного " ті держави , де захворюваність на 100 тисяч населення за останні два тижні вище , ніж в Україні , зараз цей показник виріс зі 109,5 до 126,5 за сім днів .
Так , станом на 2 жовтня , до країн червоної зони віднесені відкриті для українських туристів США , Мальдівські та Багамські острови , Ірак , Бразилія , Чорногорія , а також закриті для українських туристів Молдова , Іспанія , Франція , Чехія , Ізраїль .
У зелений список увійшли Албанія , Вірменія , Велика Британія , Сербія , Північна Македонія ,

In [27]:
%%time
news['tokenized'] = news.apply(lambda row: tokenize(row.all_text, row.language), axis=1)

CPU times: user 4min 6s, sys: 7.12 s, total: 4min 13s
Wall time: 4min 58s


In [37]:
def save_to_conllu(doc, link_hash, out_dir = '/Users/oksana/Dev/LDA_SentimentPipeline/test/conllu/'):
    out_file = out_dir + "{}.conll".format(link_hash)
    for sent_idx, sent in enumerate(doc.sents, 1):
        header = ['### ', sent_idx, link_hash]
        pd.DataFrame([header]).to_csv(out_file, sep='\t', index=False, header=None, mode='a')
        sent._.conll_pd.to_csv(out_file, index=False, sep="\t", encoding='utf-8', mode='a', header=None)
        with open(out_file, 'a') as f:
            f.write('\n')

In [34]:
news_uk = news[news.language=='uk'].copy()
news_ru = news[news.language=='ru'].copy()

In [35]:
def process_news(news, nlp=nlp_uk, start=0, finish=1000, step=100):
    for k in range(start, finish, step):
        try:
            del news_part
        except:
            pass    
        news_part = news.iloc[k:k + step].copy()
        news_part['docs'] = news_part.tokenized.apply(nlp)
        news_part.apply(lambda row: save_to_conllu(row.docs, row.hash), axis=1)

In [39]:
%%time
process_news(news_uk, nlp_uk, start=0, finish=10, step=10)

CPU times: user 19.4 s, sys: 2.48 s, total: 21.8 s
Wall time: 21.1 s


### Sentiment analysis with __[UUUSA](https://github.com/aghie/uuusa)__   ( __[article](https://arxiv.org/abs/1606.05545)__,  __[manual](http://grupolys.org/software/UUUSA/uuusa-user-manual.pdf)__ )

In [ ]:
java -Dfile.encoding=UTF-8 -jar -Xmx2g samulan-0.1.0.jar \ 
-s UkSentiData \
-r configuration_uk.xml \
-c parsed.conll \
-p samulan.properties \
-v true
-o output.txt

In [ ]:
for file in *.conll; do java -Dfile.encoding=UTF-8 -jar -Xmx2g ../samulan-0.1.1.jar \
-s ../UkSentiData \
-r ../configuration_uk.xml \
-c $file \
-p ../samulan.properties\
-sc so \
-o ../output/$file; done

In [ ]:
sentiment_files = os.listdir(sentiment_dir)
res = []
for file in sentiment_files:
    s = pd.read_csv(file, sep='\t', usecols=[0], header=None)[0]
    res.append((file.strip('.conll'), s.sum(), s.astype(str).str.cat(sep=';')))
df = pd.DataFrame(res, columns=['hash', 'sentiment', 'sent_list'])
df.hash = df.hash.astype('uint64')